In [1]:
days = 20*5

DemandC2 = [1200 1400 1400 1400 1600 0]

Mfactor = 60;
C21factor = 3;

InvC2 = zeros(days+50,1)
InvM = zeros(days+50,1)
InvC21 = zeros(days+50,1)

InvC21[1:end] = 9;

cycleC2 = 1883

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvC2[t:end] -= DemandC2[demandMonth]
        demandMonth += 1
    end
    if InvC2[t] > DemandC2[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end

    Mleft = (DemandC2[demandMonth] - InvC2[t]) * Mfactor
    C21left = (DemandC2[demandMonth] - InvC2[t]) * C21factor
    if ((InvM[t] < Mleft) || (InvC21[t] < C21left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvM[tLast:t] = InvM[tLast]
        InvC21[tLast:t] = InvC21[tLast]
        InvC2[tLast:t] = InvC2[tLast]
        
        InvM[t:end] += max(DemandC2[demandMonth]* Mfactor, cycleC2 * Mfactor * 7)
        InvC21[t:end] += max(DemandC2[demandMonth]* C21factor, cycleC2 * C21factor * 7)
    else
        if InvC2[t] > DemandC2[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end

        while ((InvM[t]-Mfactor)>0) && ((InvC21[t]-C21factor)>0) && ((t+1)<= days) && keepProduction          
            prod = round(Int64, 200*randn(1)[1] + cycleC2/2)
            nM = div(InvM[t], Mfactor)
            nC21 = div(InvC21[t], C21factor)
            
            nC2 = min(nM, nC21, prod)
            
            InvC2[t:end] += nC2
            InvM[t:end] -= nC2*Mfactor
            InvC21[t:end] -= nC2*C21factor            
            defect = rand()/10
            
            InvC2[t:end] -= round(Int64, InvC2[t+1]*defect)
            if InvC2[t] > DemandC2[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvC2[t:end] -= DemandC2[demandMonth]
                demandMonth += 1
            end
            Mleft = (DemandC2[demandMonth] - InvC2[t]) * Mfactor
            C21left = (DemandC2[demandMonth] - InvC2[t]) * C21factor
            if ((InvM[t] < Mleft) || (InvC21[t] < C21left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvM[tLast:tOrder] = InvM[tLast]
                InvC21[tLast:tOrder] = InvC21[tLast]
                InvC2[tLast:tOrder] = InvC2[tLast]
            end
            if !noOrder
                if !(((InvM[t]-Mfactor)>0) && ((InvC21[t]-C21factor)>0) && ((t+1)<= days) && keepProduction)
                    t = tOrder
                    noOrder = true
                    InvM[t:end] += max(Mleft* Mfactor, cycleC2 * Mfactor * 7)
                    InvC21[t:end] += max(C21left* C21factor, cycleC2 * C21factor * 7)
                end
                
            end
        end
    end
    t+=1
end

nOrders

1

In [2]:
using Plots
plot(1:(days+50), InvM/Mfactor, label = "M scaled",title="Inventory over 5 months")
plot!(1:(days+50), InvC21/C21factor, label="C21 scaled")
plot!(1:(days+50), InvC2, label="C2 actual")

using Images
using FileIO
savefig("c2invall.png")

[Plots.jl] Initializing backend: pyplot


In [4]:
plot(1:(days+50), InvC2, label="C2", title="Inventory of C2")

using Images
using FileIO
savefig("c2inv.png")